In [1]:
import pandas as pd
csv_file = "/Users/i337036/Downloads/train.csv"
df = pd.read_csv(csv_file, encoding='utf-8')
len(df)

404290

In [2]:
df.columns.values

array(['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate'],
      dtype=object)

In [3]:
df = df[df['is_duplicate']==1]

In [4]:
df = df.drop_duplicates(subset=['question1'], keep='first')
df = df.drop_duplicates(subset=['question2'], keep='first')
len(df)

73776

In [5]:
q1_list = df.question1.tolist()
q2_list = df.question2.tolist()
print(len(q1_list), len(q2_list))

73776 73776


In [6]:
q1q2_list = ["%s %s" % (txt1, txt2) for txt1, txt2 in zip(q1_list, q2_list)]
print(len(q1q2_list))

73776


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline

vectorizer = TfidfVectorizer(analyzer='word', min_df = 0, stop_words = 'english', use_idf=True, smooth_idf=True)
svd_model = TruncatedSVD(n_components=500, algorithm='randomized', n_iter=10, random_state=42)
svd_transformer = Pipeline([('tfidf', vectorizer), ('svd', svd_model)])
svd_matrix = svd_transformer.fit_transform(q1q2_list)

In [8]:
from sklearn.metrics.pairwise import cosine_similarity

def find_similar(index, top_n=5):
    query_vec = svd_transformer.transform([q1_list[index]])
    qd_sims = cosine_similarity(query_vec, svd_matrix).flatten()
    qd_indices = qd_sims.argsort()[:-top_n:-1]
    return qd_indices

search_idx = 100
print("Searched for %s at [%d]" % (q1_list[search_idx], search_idx))
for index in find_similar(search_idx):
    print ("%s %d" % (q2_list[index], index))

Searched for What are the rights of a prisoner? at [100]
What are the LGBT rights? 62811
If I am in prison, what rights would I have? 100
What are miranda rights and why is it called this? 73659
What are the Miranda rights? 51554


In [9]:
q1_list[search_idx], q2_list[search_idx]

('What are the rights of a prisoner?',
 'If I am in prison, what rights would I have?')

In [10]:
from tqdm import tqdm

In [11]:
correct_index = 0
for search_idx in tqdm(range(500)):
    for index in find_similar(search_idx):
        if search_idx==index or q2_list[search_idx]==q2_list[index]:
            correct_index += 1
            break

100%|██████████| 500/500 [03:14<00:00,  2.57it/s]


In [12]:
print(correct_index)

368
